In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
from tavily import TavilyClient
tavilyClient = TavilyClient()

In [19]:
from langchain.tools import tool
from typing import Dict, Any

@tool()
def search_recipes_by_ingredients(query: str) -> Dict[str, Any]:
  """Searchs recipes by ingredients from WEB"""
  return tavilyClient.search(query)

In [20]:
system_prompt = """
You are a cousine chief and helps the user to create recipes by their remanescent ingredints.

Using the search recipes by ingredients tools, search the web for recipes that can be made with the ingredients they have.

Before search for recipes, analise and check if the image contains ingredients to make food. If not, responds with : IMAGE INVALID

If yes, return recipe suggestions and eventually the recipe instructions to the user, if requested
"""

In [21]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

config = {"configurable": {"thread_id": "1"}}

agent = create_agent(
    model='gpt-5-nano',
    system_prompt=system_prompt,
    tools=[search_recipes_by_ingredients],
    checkpointer=InMemorySaver()
)

In [22]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.png', multiple=False)
display(uploader)

FileUpload(value=(), accept='.png', description='Upload')

In [23]:
import base64 

uploaded_file = uploader.value[0]

content_mv = uploaded_file["content"]

img_bytes = bytes(content_mv)

img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [24]:
from langchain.messages import HumanMessage

multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Search recipes to me based on this ingredients"},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])

response = agent.invoke(
    {"messages": [multimodal_question]},
    config
)

print(response['messages'][-1].content)

IMAGE ANALYSIS: The image shows ingredients you have: grapefruit, lemons, oranges, kiwis, fresh ginger, blueberries, almonds, yogurt, honey, turmeric. Great lineup for several fast, healthy recipes.

Here are recipe ideas that fit your ingredients, with sources. Tell me which one you’d like full step-by-step instructions for and I’ll pull them in.

1) Kick the Cold Citrus Smoothie (Deliciously Ella)
- Why it fits: uses orange, grapefruit, lemon, ginger, honey (you can add yogurt if you like for creaminess).
- Source: Deliciously Ella
- Link: https://www.facebook.com/Deliciouslyella/posts/feeling-a-little-run-down-you-need-this-kick-the-cold-smoothieloaded-with-fresh-/1174067930753200/

2) Immune-Boosting Citrus Berry Smoothie (Crowded Kitchen)
- Why it fits: blends citrus with berries and a little spice—great base for blueberries, with ginger and turmeric; yogurt and honey optional.
- Source: Crowded Kitchen
- Link: https://www.crowdedkitchen.com/immune-boosting-smoothies/

3) Wild Blu

In [25]:
## Invoking again to test memory

response = agent.invoke(
    {"messages": [multimodal_question, HumanMessage(content="Detail the first option")]},
    config
)

print(response['messages'][-1].content)

Here’s a detailed version of the first option: Kick the Cold Citrus Smoothie (inspired by Deliciously Ella). It uses your ingredients: orange, grapefruit, lemon, fresh ginger, turmeric, and honey. I’ve included dairy-free options and tips.

Kick the Cold Citrus Smoothie (inspired)

Servings: 1 large smoothie

Ingredients
- 1/2 grapefruit, peeled and if possible, peeled to remove white pith
- 1 orange, peeled
- 1/2 lemon, peeled
- 1/2 inch fresh ginger, peeled
- 1/2 inch fresh turmeric root, peeled (or 1/2 teaspoon ground turmeric)
- 1 tablespoon honey (adjust to taste; you can use maple syrup for a vegan option)
- 1/2 cup plain yogurt (d dairy) or 1/2 cup dairy-free yogurt (or use 1/4 cup yogurt + 1/4 cup almond milk if you want a lighter texture)
- 1/2 cup ice (or 1/4 cup ice + 1/4 cup water/juice)
- Optional: pinch black pepper (helps with turmeric absorption)
- Optional: a few fresh mint leaves for garnish

Instructions
1) Prep the produce: Peel and section the grapefruit and orange